#Abstract#

In this era of entertainment the companies like Netflix, Amazon prime etc. have provided a huge platform for online streaming of movies, series, documentries and other visual entertainment. The users of these platforms have always been in a dilemma to chose which kind of movies they would like to watch amongst the millions of movies uploaded by these platforms. That is where big data and recommendations have come in for analyzing and providing recommendations to its users which makes it very simple for these users to access the kind of movie acoording to the genres they are interested in.

#Motivation#

This dataset was provided for a Netflix recommendation contest which will provide an opportunity and challenges to learn different types of filtering techniques used in recommendation systems. The project is to be performed on databricks platform using PySpark.

#Introduction to the Data#

This dataset is provided by movie lens with 2 files which we will be using for this project which are ratings and movies. The movies.csv has columns which are movieId, title, genre where as the ratings.csv has userId,movieId,ratings and timestamp.

Data Source and dictonary : https://grouplens.org/datasets/movielens/

Data consistency: The data seems to be mostly clean with some missing values which have been dropped. There are a few duplicates in the data which have been dropped.

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import explode,split
from pyspark.sql.window import Window
from IPython.display import Image 
from IPython.core.display import HTML 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# File location of MOVIE dataset
movie_file_location = "/FileStore/tables/movies.csv"

In [0]:
#defining schema for Movie dataset
movie_schema = StructType([StructField('movieId', DoubleType(), True),       
                     StructField('title', StringType(), True),
                     StructField('genres', StringType(), True)])

In [0]:
#reading the data using schema definition 
moviedf = spark.read.csv(movie_file_location, header=True , schema=movie_schema)
display(moviedf)

movieId,title,genres
1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2.0,Jumanji (1995),Adventure|Children|Fantasy
3.0,Grumpier Old Men (1995),Comedy|Romance
4.0,Waiting to Exhale (1995),Comedy|Drama|Romance
5.0,Father of the Bride Part II (1995),Comedy
6.0,Heat (1995),Action|Crime|Thriller
7.0,Sabrina (1995),Comedy|Romance
8.0,Tom and Huck (1995),Adventure|Children
9.0,Sudden Death (1995),Action
10.0,GoldenEye (1995),Action|Adventure|Thriller


In [0]:
#Making Function to seperate year from movie title with Regex

import re

def sepYear(title):
  year = re.search('\(([0-9]{4})', title)
  if year is not None:
    year = year.group(1)
  else:
    year = 0

  return int(year)

In [0]:
# Registering Functions
sqlContext.udf.register("yearCleansing", sepYear)

Out[7]: <function __main__.sepYear(title)>

In [0]:
# Using The Functions

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

year_udf = udf(sepYear, IntegerType())
moviedf = moviedf.select("movieId", "title", "genres", year_udf("title").alias("year"))

In [0]:
moviedf.printSchema()

root
-- movieId: double (nullable = true)
-- title: string (nullable = true)
-- genres: string (nullable = true)
-- year: integer (nullable = true)

In [0]:
display(moviedf)

movieId,title,genres,year
1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
2.0,Jumanji (1995),Adventure|Children|Fantasy,1995
3.0,Grumpier Old Men (1995),Comedy|Romance,1995
4.0,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
5.0,Father of the Bride Part II (1995),Comedy,1995
6.0,Heat (1995),Action|Crime|Thriller,1995
7.0,Sabrina (1995),Comedy|Romance,1995
8.0,Tom and Huck (1995),Adventure|Children,1995
9.0,Sudden Death (1995),Action,1995
10.0,GoldenEye (1995),Action|Adventure|Thriller,1995


In [0]:
display(moviedf)

movieId,title,genres,year
1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
2.0,Jumanji (1995),Adventure|Children|Fantasy,1995
3.0,Grumpier Old Men (1995),Comedy|Romance,1995
4.0,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
5.0,Father of the Bride Part II (1995),Comedy,1995
6.0,Heat (1995),Action|Crime|Thriller,1995
7.0,Sabrina (1995),Comedy|Romance,1995
8.0,Tom and Huck (1995),Adventure|Children,1995
9.0,Sudden Death (1995),Action,1995
10.0,GoldenEye (1995),Action|Adventure|Thriller,1995


In [0]:
#checking if there are any duplicate values of the title.
display(moviedf.groupby("title").count().orderBy("title", ascending=False))

title,count
줄탁동시 (2012),1
貞子3D (2012),1
チェブラーシカ (2010),1
キサラギ (2007),1
…And the Fifth Horseman Is Fear (1965),1
‘Rameau’s Nephew’ by Diderot (Thanx to Dennis Young) by Wilma Schoen (1974),1
काशी - In Search of Ganga (2018),1
به نام پدر,1
أهواك (2015),1
ארבינקא (1967),1


In [0]:
# Counting the rows of the movies dataframe.
moviedf.count()

Out[13]: 62423

In [0]:
#Summary of the statistics of Movie dataset.
display(moviedf.select('movieId').summary())

summary,movieId
count,62423
mean,122220.38764557935
stddev,63264.74484425327
min,1.0
25%,82121.0
50%,138010.0
75%,173215.0
max,209171.0


In [0]:
# File location of MOVIE RATING dataset
rating_file_location = "/FileStore/tables/ratings.csv"

In [0]:
#Defining the schmea for the Rating dataframe.
rating_schema = StructType([StructField('userId', DoubleType(), True),        
                     StructField('movieId', DoubleType(), True),
                     StructField('rating', DoubleType(), True),
                     StructField('timestamp', DoubleType(), True)])

In [0]:
#Reading the dataset using schema definition.
ratingdf = spark.read.csv(rating_file_location, header=True, schema=rating_schema )
display(ratingdf)

userId,movieId,rating,timestamp
1.0,296.0,5.0,1.147880044E9
1.0,306.0,3.5,1.147868817E9
1.0,307.0,5.0,1.147868828E9
1.0,665.0,5.0,1.14787882E9
1.0,899.0,3.5,1.14786851E9
1.0,1088.0,4.0,1.147868495E9
1.0,1175.0,3.5,1.147868826E9
1.0,1217.0,3.5,1.147878326E9
1.0,1237.0,5.0,1.147868839E9
1.0,1250.0,4.0,1.147868414E9


In [0]:
display(ratingdf)

userId,movieId,rating,timestamp
1.0,296.0,5.0,1.147880044E9
1.0,306.0,3.5,1.147868817E9
1.0,307.0,5.0,1.147868828E9
1.0,665.0,5.0,1.14787882E9
1.0,899.0,3.5,1.14786851E9
1.0,1088.0,4.0,1.147868495E9
1.0,1175.0,3.5,1.147868826E9
1.0,1217.0,3.5,1.147878326E9
1.0,1237.0,5.0,1.147868839E9
1.0,1250.0,4.0,1.147868414E9


In [0]:
#No. of rows in the dataframe.
ratingdf.count()

Out[19]: 25000095

In [0]:
#Checking the statistical summary for the Ratings dataset
display(ratingdf.select('movieId').summary())

summary,movieId
count,25000095
mean,21387.981943268616
stddev,39198.86210105984
min,1.0
25%,1197.0
50%,2947.0
75%,8623.0
max,209171.0


In [0]:
#Dropping the columns not required for the current project.
ratingdf = ratingdf.drop('timestamp')

In [0]:
#grouping movies according to its genres
display(moviedf.groupby("genres").count().orderBy("count", ascending=False))

genres,count
Drama,9056
Comedy,5674
(no genres listed),5062
Documentary,4731
Comedy|Drama,2386
Drama|Romance,2126
Horror,1661
Comedy|Romance,1577
Comedy|Drama|Romance,1044
Drama|Thriller,933


In [0]:
#joining the dataframes.
movie_rating = (ratingdf.select('userId','movieId','rating')).join(moviedf,['movieId'])
display(movie_rating)

movieId,userId,rating,title,genres,year
296.0,1.0,5.0,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1994
306.0,1.0,3.5,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama,1994
307.0,1.0,5.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama,1993
665.0,1.0,5.0,Underground (1995),Comedy|Drama|War,1995
899.0,1.0,3.5,Singin' in the Rain (1952),Comedy|Musical|Romance,1952
1088.0,1.0,4.0,Dirty Dancing (1987),Drama|Musical|Romance,1987
1175.0,1.0,3.5,Delicatessen (1991),Comedy|Drama|Romance,1991
1217.0,1.0,3.5,Ran (1985),Drama|War,1985
1237.0,1.0,5.0,"Seventh Seal, The (Sjunde inseglet, Det) (1957)",Drama,1957
1250.0,1.0,4.0,"Bridge on the River Kwai, The (1957)",Adventure|Drama|War,1957


In [0]:
#caching the dataframes.
movie_rating.cache
ratingdf.cache

Out[24]: <bound method DataFrame.cache of DataFrame[userId: double, movieId: double, rating: double]>

In [0]:
#printing the joined dataframe schema.
movie_rating.printSchema()

root
-- movieId: double (nullable = true)
-- userId: double (nullable = true)
-- rating: double (nullable = true)
-- title: string (nullable = true)
-- genres: string (nullable = true)
-- year: integer (nullable = true)

In [0]:
#Dropping null values
movie_rating = movie_rating.dropna()

In [0]:
#Dropping Duplicate entries.
movie_rating = movie_rating.dropDuplicates() 

In [0]:
#checking the number of rows.
movie_rating.count()

Out[28]: 25000095

In [0]:
#grouping movies according to its genres
display(moviedf.groupby("genres").count().orderBy("count", ascending=False))

genres,count
Drama,9056
Comedy,5674
(no genres listed),5062
Documentary,4731
Comedy|Drama,2386
Drama|Romance,2126
Horror,1661
Comedy|Romance,1577
Comedy|Drama|Romance,1044
Drama|Thriller,933


In [0]:
from pyspark.sql.functions import explode,split
genres = moviedf.withColumn("genres",explode(split("genres","[|]")))
display(genres)

movieId,title,genres,year
1.0,Toy Story (1995),Adventure,1995
1.0,Toy Story (1995),Animation,1995
1.0,Toy Story (1995),Children,1995
1.0,Toy Story (1995),Comedy,1995
1.0,Toy Story (1995),Fantasy,1995
2.0,Jumanji (1995),Adventure,1995
2.0,Jumanji (1995),Children,1995
2.0,Jumanji (1995),Fantasy,1995
3.0,Grumpier Old Men (1995),Comedy,1995
3.0,Grumpier Old Men (1995),Romance,1995


In [0]:
genre = genres.select('genres').drop_duplicates()
display(genre)

genres
Crime
Romance
Thriller
Adventure
Drama
War
Documentary
Fantasy
Mystery
Musical


As we can see that there are multiple genres for same movieId and title. Combining these genres into one. Providing ranks for measuring the ratings.

In [0]:
from pyspark.sql.window import Window
movie_rating= movie_rating.withColumn("rank",row_number().over(Window.partitionBy("userId","title").orderBy("rating")))
movie_rating.show()

+--------+------+------+--------------------+--------------------+----+----+
 movieId|userId|rating| title| genres|year|rank|
+--------+------+------+--------------------+--------------------+----+----+
 31956.0| 1.0| 3.5| 5x2 (2004)| Drama|Romance|2004| 1|
 6539.0| 1.0| 3.5|Pirates of the Ca...|Action|Adventure|...|2003| 1|
 899.0| 1.0| 3.5|Singin' in the Ra...|Comedy|Musical|Ro...|1952| 1|
 3510.0| 2.0| 4.0| Frequency (2000)| Drama|Thriller|2000| 1|
 4886.0| 2.0| 5.0|Monsters, Inc. (2...|Adventure|Animati...|2001| 1|
 534.0| 2.0| 4.5| Shadowlands (1993)| Drama|Romance|1993| 1|
 1210.0| 2.0| 5.0|Star Wars: Episod...|Action|Adventure|...|1983| 1|
 2987.0| 2.0| 0.5|Who Framed Roger ...|Adventure|Animati...|1988| 1|
 5810.0| 3.0| 3.5| 8 Mile (2002)| Drama|2002| 1|
 94018.0| 3.0| 2.5| Battleship (2012)|Action|Sci-Fi|Thr...|2012| 1|
115617.0| 3.0| 4.0| Big Hero 6 (2014)|Action|Animation|...|2014| 1|
 8798.0| 3.0| 3.5| Collateral (2004)|Action|Crime|Dram...|2004| 1|
 99114.0| 3.0| 4.0|Django Unchained ...|Action|Drama|Western|2012| 1|
 4019.0| 3.0| 4.0|Finding Forrester...| Drama|2000| 1|
136449.0| 3.0| 5.0|Ghost in the Shel...|Action|Animation|...|2008| 1|
 97225.0| 3.0| 3.5|Hotel Transylvani...|Animation|Childre...|2012| 1|
 47610.0| 3.0| 4.0|Illusionist, The ...|Drama|Fantasy|Mys...|2006| 1|
 6564.0| 3.0| 3.5|Lara Croft Tomb R...|Action|Adventure|...|2003| 1|
 96610.0| 3.0| 4.0| Looper (2012)| Action|Crime|Sci-Fi|2012| 1|
 36529.0| 3.0| 4.0| Lord of War (2005)|Action|Crime|Dram...|2005| 1|
+--------+------+------+--------------------+--------------------+----+----+
only showing top 20 rows

In [0]:
movie_rating.filter(movie_rating.rank==1).show()

+--------+------+------+--------------------+--------------------+----+----+
 movieId|userId|rating| title| genres|year|rank|
+--------+------+------+--------------------+--------------------+----+----+
 1175.0| 1.0| 3.5| Delicatessen (1991)|Comedy|Drama|Romance|1991| 1|
 6370.0| 1.0| 4.5|Spanish Apartment...|Comedy|Drama|Romance|2002| 1|
 2745.0| 2.0| 5.0| Mission, The (1986)| Drama|1986| 1|
 1374.0| 2.0| 4.5|Star Trek II: The...|Action|Adventure|...|1982| 1|
 6333.0| 2.0| 5.0|X2: X-Men United ...|Action|Adventure|...|2003| 1|
 78469.0| 3.0| 3.0| A-Team, The (2010)|Action|Comedy|Thr...|2010| 1|
 37386.0| 3.0| 4.0| Aeon Flux (2005)| Action|Sci-Fi|2005| 1|
 55805.0| 3.0| 4.0|Before the Devil ...|Crime|Drama|Thriller|2007| 1|
 912.0| 3.0| 4.5| Casablanca (1942)| Drama|Romance|1942| 1|
 2959.0| 3.0| 5.0| Fight Club (1999)|Action|Crime|Dram...|1999| 1|
 73323.0| 3.0| 4.0|Girl Who Kicked t...|Action|Crime|Mystery|2009| 1|
 88125.0| 3.0| 3.5|Harry Potter and ...|Action|Adventure|...|2011| 1|
 69844.0| 3.0| 3.5|Harry Potter and ...|Adventure|Fantasy...|2009| 1|
 57640.0| 3.0| 4.0|Hellboy II: The G...|Action|Adventure|...|2008| 1|
 93363.0| 3.0| 4.0| John Carter (2012)|Action|Adventure|...|2012| 1|
 4027.0| 3.0| 4.5|O Brother, Where ...|Adventure|Comedy|...|2000| 1|
103228.0| 3.0| 3.0| Pacific Rim (2013)|Action|Adventure|...|2013| 1|
 3969.0| 3.0| 4.0|Pay It Forward (2...| Drama|2000| 1|
 7163.0| 3.0| 3.5| Paycheck (2003)|Action|Sci-Fi|Thr...|2003| 1|
 71530.0| 3.0| 3.5| Surrogates (2009)|Action|Sci-Fi|Thr...|2009| 1|
+--------+------+------+--------------------+--------------------+----+----+
only showing top 20 rows

In [0]:
display(movie_rating.sort("Rating").select("title","Rating"))

title,Rating
Terminator 2: Judgment Day (1991),0.5
"Princess Diaries, The (2001)",0.5
No Direction Home: Bob Dylan (2005),0.5
"Hobbit: The Desolation of Smaug, The (2013)",0.5
Sirens (1994),0.5
Kung Fu Hustle (Gong fu) (2004),0.5
"Day After Tomorrow, The (2004)",0.5
Die Another Day (2002),0.5
Batman (1989),0.5
Who Framed Roger Rabbit? (1988),0.5


In [0]:
import pyspark.sql.functions as F

(movie_rating
.groupBy(F.col('title'))
.agg(F.count('title').alias('No_of_person_viewed'))
.sort('No_of_person_viewed',ascending=False)
.show())

+--------------------+-------------------+
 title|No_of_person_viewed|
+--------------------+-------------------+
 Forrest Gump (1994)| 81491|
Shawshank Redempt...| 81482|
 Pulp Fiction (1994)| 79672|
Silence of the La...| 74127|
 Matrix, The (1999)| 72674|
Star Wars: Episod...| 68717|
Jurassic Park (1993)| 64144|
Schindler's List ...| 60411|
 Braveheart (1995)| 59184|
 Fight Club (1999)| 58773|
Terminator 2: Jud...| 57379|
Star Wars: Episod...| 57361|
 Toy Story (1995)| 57309|
Lord of the Rings...| 55736|
Usual Suspects, T...| 55366|
Star Wars: Episod...| 54917|
Raiders of the Lo...| 54675|
American Beauty (...| 53689|
Godfather, The (1...| 52498|
Lord of the Rings...| 51138|
+--------------------+-------------------+
only showing top 20 rows

In [0]:
#Dividing the dataset for Train and Test with random seed.
(train, test) = ratingdf.randomSplit([0.7, 0.3], seed = 2020)

In [0]:
Image(url= "https://miro.medium.com/max/700/1*EIBIiW2YiakP1ftxwPF8LA.png")

Out[47]:

In [0]:
Image(url= "https://miro.medium.com/max/700/1*7_JHQ6-1nyHoB2ux1h0ZKw.png")

Out[48]:

Collaborative Filtering uses aggregation of the past behaviours of all the users and recommends items to users based on the items liked by other users who have likes and dislikes that are similar to the user who is under consideration. This is called user-user based CF.

#Recommendation using ALS.#

Alternating Least Squares (ALS) matrix factorization is used to estimate the matrix R (Ratings in this case) which is the product of two matrices that are lower rank X and Y where X * Yt = R.


The method is iterative and in each iteration one of the factor matrices is kept constant and other is solved using least squares. Then the newly solved matrix is held constant and the other factor matrix is solved.

As we are looking to build a model explictly we will specify the columns. The non-negative will be true as we are looking for ratings which are greater than zero. Since we are just working with the explicit ratings we will set the implicitPrefs as False.

In [0]:
# Importing the required functions.
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Create ALS model
Als_model = ALS(
         userCol="userId", 
         itemCol="movieId",
         ratingCol="rating", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="drop"
)
# To confirm that that model was created.
type(Als_model)

Out[49]: pyspark.ml.recommendation.ALS

Defining the tuning parameter using param_grid function. Please Feel free to experiment with these parameters for the grid. 
We have only chosen 4 parameters for each grid. This will result in 16 models for training.

In [0]:
# Adding hyperparameters and respective values to the grid.
param_grid = ParamGridBuilder() \
            .addGrid(Als_model.rank, [10, 50, 100, 150]) \
            .addGrid(Als_model.regParam, [.01, .05, .1, .15]) \
            .build()

Defining Evaluator where metric is RMSE and printing length of evaluator

In [0]:
Evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="rating", 
           predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Num models to be tested: 16

Fitting the evaluator and the param_grid for ALS model into the cross validator. The number of folds we have chosen is 2. Feel free to experiment.

In [0]:
# Build cross validation using CrossValidator
Crossvalidator = CrossValidator(estimator=Als_model, estimatorParamMaps=param_grid, evaluator=Evaluator, numFolds=2)
Crossvalidator

Out[52]: CrossValidator_80777963d44c

We have chosen 16 models so it will fit and give us the best model parameters.

In [0]:
#Fit cross validator to the 'train' dataset
model = Crossvalidator.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

Printing the best model and its parameters

In [0]:
# Print best_model-
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2537899717241142> in <module> 
 1 # Print best_model- 
 ----> 2 print ( type ( best_model ) ) 
 3 
 4 # Complete the code below to extract the ALS model parameters 
 5 print ( "**Best Model**" ) 

 NameError : name 'best_model' is not defined

Calculating and evaluating by the metric RMSE with the best model.

In [0]:
# View the predictions
test_data_prediction = best_model.transform(test)
RMSE = Evaluator.evaluate(test_data_prediction)
print(RMSE)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2537899717241149> in <module> 
 1 # View the predictions 
 ----> 2 test_data_prediction = best_model . transform ( test ) 
 3 RMSE = Evaluator . evaluate ( test_data_prediction ) 
 4 print ( RMSE ) 

 NameError : name 'best_model' is not defined

Displaying the predicted values from the test dataset.

In [0]:
test_data_prediction.show()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2537899717241143> in <module> 
 ----> 1 test_data_prediction . show ( ) 

 NameError : name 'test_data_prediction' is not defined

Predicting Recommendations for all these users.

In [0]:
# Generate n Recommendations for all users
n_recommendations = best_model.recommendForAllUsers(10)
n_recommendations.show()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2537899717241151> in <module> 
 1 # Generate n Recommendations for all users 
 ----> 2 n_recommendations = best_model . recommendForAllUsers ( 10 ) 
 3 n_recommendations . show ( ) 

 NameError : name 'best_model' is not defined

Transforming the dataframe into a better understandable format.

In [0]:
n_recommendations = n_recommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))
n_recommendations.limit(10).show()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2537899717241152> in <module> 
 ----> 1 n_recommendations = n_recommendations \ 
 2 . withColumn ( "rec_exp" , explode ( "recommendations" ) ) \ 
 3 . select ( 'userId' , col ( "rec_exp.movieId" ) , col ( "rec_exp.rating" ) ) 
 4 n_recommendations . limit ( 10 ) . show ( ) 

 NameError : name 'n_recommendations' is not defined

Lets check if these recommendations make any sense? We will print the original genres of movies watched by the userId 50 and we will check if the predicted movies are of similar genre.

In [0]:
n_recommendations.join(moviedf, on='movieId').filter('userId = 50').show()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2537899717241153> in <module> 
 ----> 1 n_recommendations . join ( moviedf , on = 'movieId' ) . filter ( 'userId = 50' ) . show ( ) 

 NameError : name 'n_recommendations' is not defined

In [0]:
ratingdf.join(moviedf, on='movieId').filter('userId = 50').sort('rating', ascending=False).limit(10).show()

+-------+------+------+--------------------+--------------------+----+
movieId|userId|rating| title| genres|year|
+-------+------+------+--------------------+--------------------+----+
 1645.0| 50.0| 5.0|The Devil's Advoc...|Drama|Mystery|Thr...|1997|
 4025.0| 50.0| 5.0|Miss Congeniality...| Comedy|Crime|2000|
 1784.0| 50.0| 5.0|As Good as It Get...|Comedy|Drama|Romance|1997|
 344.0| 50.0| 5.0|Ace Ventura: Pet ...| Comedy|1994|
 2329.0| 50.0| 5.0|American History ...| Crime|Drama|1998|
 364.0| 50.0| 5.0|Lion King, The (1...|Adventure|Animati...|1994|
 2541.0| 50.0| 5.0|Cruel Intentions ...| Drama|1999|
 500.0| 50.0| 5.0|Mrs. Doubtfire (1...| Comedy|Drama|1993|
 2881.0| 50.0| 5.0|Double Jeopardy (...|Action|Crime|Dram...|1999|
 1393.0| 50.0| 5.0|Jerry Maguire (1996)| Drama|Romance|1996|
+-------+------+------+--------------------+--------------------+----+

#Conclusion and Results#

We can see that the movies which were originally viewed by the 50th userId and the movies predicted by this model are of the similar genre which is War, Drama etc. Thus, we can verify the the values our model is predicting are very similar.

#Future Work and Limitations#
In this project we have used some param_grid inputs and 16 models. There can be more number models that can be experimented for a better model and we can see the different results they display. In this project we have only used ALS where as there are several other recommendation methods that can be explored.

#References and Related work#

1. https://grouplens.org/datasets/movielens/
2. http://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html
3. https://www.data-stats.com/spark-analytics-on-movielens-dataset/
4. https://intellipaat.com/community/5161/how-do-i-add-a-new-column-to-a-spark-dataframe-using-pyspark
5. https://en.wikipedia.org/?title=Collaborative_filtering#/media/File:Collaborative_filtering.gif
6. https://github.com/snehalnair/als-recommender-pyspark/blob/master/Recommendation_Engine_MovieLens.ipynb 
7. https://dl.acm.org/doi/10.1109/MC.2009.263
8. https://forums.databricks.com/questions/29012/how-to-show-an-image-in-a-notebook-using-html.html 
9. https://developers.google.com/machine-learning/recommendation/collaborative/matrix